In [2]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append('../')
sys.path.append('/usr/users/fsimone/tejaas')
import pandas as pd
import numpy as np
import operator
import json
from utils import utils
json_file = "../gtex_v8_metadata.json"
tissue_file = "../plots/tissues.txt"
tshorts, tfulls_plain = utils.read_tissues(tissue_file, plain=True)
_, tfulls = utils.read_tissues(tissue_file)
with open(json_file) as instream:
    gtex_meta = json.load(instream)
tissue_colors  = dict()
tissue_names   = dict()
tissue_samples = dict()
for tshort, tfull in zip(tshorts, tfulls_plain):
    tissue_names[tshort] = tfull
for tshort, tfull in zip(tshorts, tfulls):
    tissue_colors[tshort] = "#" + gtex_meta[tfull.replace(" ", "_")]["colorHex"]
    tissue_samples[tshort] = gtex_meta[tfull.replace(" ", "_")]["rnaSeqAndGenotypeSampleCount"]

sorted_tissues = [x[0] for x in sorted(tissue_samples.items(), key=operator.itemgetter(1))]
select_tissues = ['ms', 'wb', 'sse', 'as', 'hlv', 'colt', 'bco', 'aa', 'ag' , 'ac', 'bca', 'ac', 'fib', 'bceh', 'bhi', 'tes', 'nt', 'thy']


In [3]:
DATA = pd.read_csv("multi_r2_ALL.txt", header=0, index_col=0, sep="\t")


In [4]:
import fastcluster
from scipy.cluster import hierarchy

def calculate_linkage(matrix):
    data = matrix
    row_linkage = fastcluster.linkage_vector(data.fillna(0).values,
                               method='single',
                               metric="euclidean")
#     row_linkage = hierarchy.linkage(data.fillna(0), method="average", metric='euclidean')

    row_dendrogram = hierarchy.dendrogram(row_linkage, no_plot=True, color_threshold=-np.inf)
    row_reordered_ind = row_dendrogram['leaves']

    data = matrix.T
    col_linkage = fastcluster.linkage_vector(data.fillna(0).values,
                               method='single',
                               metric="euclidean")
#     col_linkage = hierarchy.linkage(data.fillna(0), method="average", metric='euclidean')
    
    col_dendrogram = hierarchy.dendrogram(col_linkage, no_plot=True, color_threshold=-np.inf)
    col_reordered_ind = col_dendrogram['leaves']
    return matrix.iloc[row_reordered_ind, col_reordered_ind]

In [1]:
import matplotlib
# matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns; sns.set()

clusterDATA = calculate_linkage(DATA).T

# fig = plt.figure(figsize=(14,17))  
fig = plt.figure(figsize=(20,8), dpi=300)  
ax  = fig.add_subplot(111)
# cbar_ax = fig.add_axes([.905, .3, .05, .3])
cmap = plt.get_cmap("Blues")
# cmap = plt.get_cmap("Spectral_r")

mymask = pd.isna(clusterDATA)
# cmap.set_bad("gray")

h = sns.heatmap(clusterDATA,
                cmap=cmap, #"RdBu_r",
                robust=True,
                linewidths=.5,
                cbar_kws={'label': '$R^2$', 'shrink':0.35,
                          #'ticks':[-2.0,-1.0, 0.0, 1.0, 2.0, 3.0, 4.0],
                          'extend':'both',
                          'pad':0.01,
                          'aspect':15},
                ax=ax,
                mask=mymask)
#                 cbar_ax = cbar_ax)
# h.set_facecolor('white')


# cbar=plt.colorbar(ax)
# cbar.set_yticklabels(["a","a","a","a","a","a"])
# h.figure.axes[1].set_yticklabels(["-2","-1","0","1","2","3","> 4"])

# ylabels = ["3' overlapping ncRNA", "Transcription Factors", "Processed Transcript", "Sense Intronic", "Sense Overlapping", "Protein Coding", "Antisense", "lincRNA", "", "Cis-eQTL Enrichment", "GTEx cis-eQTLs", "GTEx cis-sQTLs"]
# for l in DATA.index:
#     if l == "tmp":
#         ylabels.append("")
#     else:
#         ylabels.append(l)
# ax.set_yticklabels(ylabels, rotation="horizontal")
ax.set_yticklabels(clusterDATA.index, rotation=90) #, ha='right')

# cbar = ax.figure.colorbar(im, ax=ax)
# cbar.ax.set_ylabel("AUC", rotation=-90, va="bottom")

my_colors=[tissue_colors[t] for t in clusterDATA.index]
ax.set_aspect('equal')
for t, X in zip(clusterDATA.index, ax.get_yticks()):
    ax.add_patch(patches.Circle((-0.5, X), radius=.3,
                                 fill=True,clip_on=False, color=tissue_colors[t]))

labels = ax.get_yticklabels()
ax.set_yticklabels([tissue_names[l.get_text()] for l in labels], rotation='horizontal', ha='right')    
    
# labels = ax.get_yticklabels()
# for label in labels:
#     print(label.get_position())
#     label.set_y(label.get_position()[1] + 0.3)
#     print(label.get_position())

plt.show()
# plt.savefig("heatmap.png")

NameError: name 'calculate_linkage' is not defined

In [ ]:
new_index = list()
cov_color = list()
for i in knn_cov_r2.index:
    if i in sample_cov_dict:
        new_index.append(sample_cov_dict[i])
        cov_color.append("#ff9f19")
    elif i in cov_dict:
        new_index.append(cov_dict[i])
        cov_color.append("#26d18a")
    else:
        new_index.append(i)
        cov_color.append("#9b26d1")
new_index

In [ ]:
import seaborn as sns; sns.set()

tmp_df = knn_cov_r2.T.copy(deep=True)
tmp_df.index = [tissue_names[x] for x in tmp_df.index]
tmp_df.columns = new_index
# tmp_df.columns = [tissue_names[x] for x in tmp_df.columns]
# tmp_df.index = new_index

# fig, (cax, ax) = plt.subplots(nrows=2, figsize=(28,14), gridspec_kw={"height_ratios":[0.025,1]})

sns.set_context("paper", font_scale=1.8)
g = sns.clustermap(tmp_df.fillna(0),
                   row_colors = [tissue_colors[x] for x in knn_cov_r2.columns], 
                   col_colors = cov_color,
                   mask = pd.isna(tmp_df),
                   figsize=(30,8),
                   cmap="Blues",
                   linewidths=.95,
                   cbar_kws={"orientation": "vertical", "label":"R²"},
                   robust=True ) #col_cluster=False)
g.ax_row_dendrogram.set_visible(False)
#g.gs.set_height_ratios([0.1,0.1,0.5])
g.cax.set_visible(False)
#g.ax_row_dendrogram.set_axis_off()
#g.cax.set_ax(g.ax_row_dendrogram.axes)
#cg.ax_row_dendrogram.set_xlim([0,0])
#g.ax_row_colors.set_alpha(0.9)
s = g.ax_heatmap.set_xticklabels([i for i in g.ax_heatmap.get_xticklabels()], rotation=45, ha='right')
# g.savefig("knn_clustermap_rotated_labels.png")
#g.gs

In [ ]:
import matplotlib 

def fixedWidthClusterMap(dataFrame, cellSizePixels=50):
    # Calulate the figure size, this gets us close, but not quite to the right place
    dpi = matplotlib.rcParams['figure.dpi']
    marginWidth = matplotlib.rcParams['figure.subplot.right']-matplotlib.rcParams['figure.subplot.left']
    marginHeight = matplotlib.rcParams['figure.subplot.top']-matplotlib.rcParams['figure.subplot.bottom']
    Ny,Nx = dataFrame.shape
    figWidth = (Nx*cellSizePixels/dpi)/0.8/marginWidth
    figHeigh = (Ny*cellSizePixels/dpi)/0.8/marginHeight

    # sns.set_context("paper", font_scale=1.8)
    grid = sns.clustermap(dataFrame.fillna(0),
                       row_colors = [tissue_colors[x] for x in knn_cov_r2.columns], 
                       col_colors = cov_color,
                       mask = pd.isna(dataFrame),
                       figsize=(figWidth, figHeigh),
                       cmap="Blues",
                       linewidths=.95,
                       cbar_kws={"orientation": "vertical", "label":"R²"},
                       robust=True ) #col_cluster=False)
    grid.ax_row_dendrogram.set_visible(False)
    grid.cax.set_visible(False)
#     grid.ax_col_dendrogram.set_visible(False)
    s = grid.ax_heatmap.set_xticklabels([i for i in g.ax_heatmap.get_xticklabels()], rotation=45, ha='right')

    # calculate the size of the heatmap axes
    axWidth = (Nx*cellSizePixels)/(figWidth*dpi)
    axHeight = (Ny*cellSizePixels)/(figHeigh*dpi)
    
    print(axWidth, axHeight)
    # resize dendrograms to match
    ax_row_orig_pos = grid.ax_row_dendrogram.get_position()
#     grid.ax_row_dendrogram.set_position([ax_row_orig_pos.x0, ax_row_orig_pos.y0, 
#                                          ax_row_orig_pos.width, axHeight])
    ax_col_orig_pos = grid.ax_col_dendrogram.get_position()
#     grid.ax_col_dendrogram.set_position([ax_col_orig_pos.x0, ax_heatmap_orig_pos.y0+axHeight,
#                                          axWidth, ax_col_orig_pos.height])
    
#     ax_colcolors_orig_pos = grid.ax_col_colors.get_position()
#     grid.ax_col_colors.set_position(ax_colcolors_orig_pos.x0, ax_colcolors_orig_pos.y0, 
#                                     axWidth, ax_colcolors_orig_pos+axHeight)
    
#     print(grid.ax_col_colors.get_position())
#     # resize heatmap
#     ax_heatmap_orig_pos = grid.ax_heatmap.get_position()
# #     grid.ax_heatmap.set_position([ax_heatmap_orig_pos.x0, ax_heatmap_orig_pos.y0, 
# #                                   axWidth, axHeight])
#     grid.ax_heatmap.set_position([ax_col_orig_pos.x0, ax_heatmap_orig_pos.y0, 
#                                   axWidth, axHeight])
    return grid
    
grid = fixedWidthClusterMap(tmp_df, cellSizePixels=100)
grid.savefig("clustermap_squared_knn.png")
plt.show()

In [ ]:
import seaborn as sns; sns.set()

tmp_df = knn_cov_r2.T.copy(deep=True)
tmp_df.index = [tissue_names[x] for x in tmp_df.index]
tmp_df.columns = new_index

vmin = np.percentile(tmp_df.values, 2)
vmax = np.percentile(tmp_df.values, 98)

sns.set_context("paper", font_scale=1.8)
g = sns.heatmap(tmp_df,
                cmap="Blues",
                linewidths=.75,
                cbar_kws={"orientation": "vertical", "label":"R²"},
                robust=True ) #col_cluster=False)
